# Trabajo Práctico N2 Aprendizaje Automático

## Integrantes
#### Federico Canay
#### Diego Raffo

## Introduccion

En este trabajo planteamos como objetivo poder discriminar imágenes y categorizarlas como perro o gato, si es que en ellas aparecía un perro o un gato respectivamente. Para esto se contó con un dataset de prueba de 25.000 imágenes.
Para esto se desarrolló una herramienta que a partir de una muestra de imágenes ya categorizadas, intenta discriminar nuevas imágenes no categorizadas,  asignándolas al grupo de perros o de gatos. 

El desarrollo se hizo en python, usando bibliotecas numéricas como numpy y scipy y bibliotecas de aprendizaje automático como sklearn, openCV y mahotas.


## Sesión 1: Extracción de Atributos 

Al trabajar con un problema de imágenes se agrega un paso extra y de suma importancia para generar un clasificador, la extracción de atributos.

No se puede ingresar como input de un algoritmo de ML una imagen (la representación matricial de sus pixeles en escala de grises o RGB), ya que imágenes de diferentes resoluciones, tendrían diferentes cantidad de atributos generando que solo pueda clasificar imagenes de cierta resolución (además de necesitar suficientes imágenes de esa resolución para poder entrenarlo). A esto se le suma que las imágenes tienen una semántica espacial, al representar la imagen como un vector de pixeles, estamos perdiendo información. Ya que un pixel no solo depende de el pixel a su derecha e izquierda sino también de los de su alrededor.
    
Por esta extraemos atributos de una imagen y este será el input de nuestros algoritmos.

Es importante recalcar que la cantidad de atributos tiene que ser independiente de la imagen (o sea, a toda imagen se le debe asignar la misma cantidad de atributos).

Para trabajar las imágenes usamos opencv2

Cargamos una imagen:

In [1]:
image = cv2.imread(imageFile)

NameError: name 'cv2' is not defined

Y cuando generamos atributos sobre imágenes blanco y negro, transformamos la imagen a blanco y negro de la siguiente manera:

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

### Histogramas

Como primera aproximación decidimos generar un histograma de la distribución de grises y de colores en la imagen.
En el caso de escala de grises se genera un vector de 256 componentes, cada uno corresponde a la cantidad de pixeles de ese valor de la escala de grises.

Generación de histograma blanco y negro

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hist,bins = np.histogram(image.flatten(),256,[0,256])
return hist

El caso de color es muy parecido solo que generamos un histograma para cada color de la descomposición RGB.

In [ ]:
b,g,r = cv2.split(image)
b_hist,bins = np.histogram(b.flatten(),256,[0,256])
g_hist,bins = np.histogram(g.flatten(),256,[0,256])
r_hist,bins = np.histogram(r.flatten(),256,[0,256])
return np.concatenate((b_hist,g_hist,r_hist))

Aunque es una primera es una primera aproximación, nuestra hipótesis es que no tenga un gran desempeño ya que tanto perros y gatos suelen tener colores similares y suponemos que entonces la distribución de los mismas no servirá para distinguirlos.

### Patrones

Intentamos buscar patrones de texturas dentro de cada imagenes, desde algunos simples y generales a otros un poco más específicos. Todos estos patrones se hicieron en blanco y negro, ya que el número de posibles patrones crece exponencialmente y se vuelve inmanejable al considerar más de 2 estados para cada color (claro y oscuro). Al armar los patrones definimos como un pixel claro (0) a los pixels cuya escala de grises sea inferior a 128, y oscuros (1) al resto.

In [ ]:
def oscuro(gris):
    if gris > 127:
        return 1
    else:
        return 0

Nuestra idea es que estos patrones logren identificar las texturas características de tanto perros como gatos

 #### ’2x2’ y ‘3x3’

Los patrones de 2 x 2 son patrones en grupos cuadrados de 4 pixeles donde se buscan todas las combinaciones posibles de pixeles claros y oscuros, como los acabamos de definir. El diccionario de patrones se arma de la siguiente manera. Hay 16 posibles patrones (ya que cada pixel tiene dos opciones), así tendremos 16 atributos, que representará la cantidad de veces que encontramos ese patrón en la imagen


In [ ]:
greyImage = cv2.cvtColor(greyImage, cv2.COLOR_BGR2GRAY)
patterns_dict = {'0000':0,'0001':0,'0010':0,'0100':0,'1000':0,'0011':0,'0101':0,'0110':0,'0111':0,'1001':0,'1010':0,'1011':0,'1100':0,'1101':0,'1110':0,'1111':0}
for i in xrange(greyImage.shape[0]-1):
    for j in xrange(greyImage.shape[1]-1):
        pattern = str(oscuro(greyImage[i,j])) + str(oscuro(greyImage[i+1,j]))+ str(oscuro(greyImage[i,j+1]))+ str(oscuro(greyImage[i+1,j+1]))
        if pattern in patterns_dict:
            patterns_dict[pattern] += 1
        else:
            patterns_dict[pattern] = 1

A partir de el diccionario creado se pueden obtener la cantidad de apariciones de cada patrón en la imagen analizada.

El caso de los patrones de 3x3 es bastante similar, solo que los grupos de pixeles cuadrados es de 9 pixeles y no de 4. Esto genera 512 patrones posibles, y como en el caso anterior 512 atributos, donde cada atributo representa la cantidad de veces que se encontró ese patrón en la imagen.


In [ ]:
greyImage = cv2.cvtColor(greyImage, cv2.COLOR_BGR2GRAY)
patterns_dict = [0] * 512
for i in xrange(greyImage.shape[0]-2):
    for j in xrange(greyImage.shape[1]-2):
        pattern = 0
        for x in xrange(0,3):
            for y in xrange(0,3):
                pattern += oscuro(greyImage[i+x,j+y]) * (2**(y+(3*x)))
                patterns_dict[pattern] += 1

Usar patrones más grandes nos da un poco más de especificidad, y de noción de textura, que podría resultar mejor para poder distinguir perros de gatos.

Usar este tipo de patrones todavía más grande se vuelve impracticable, ya que si tomamos cuadrados de 5x5, tendrán 25 pixeles generando 2^25 posibles patrones aprox 35 millones. lo que generaría 35 millones de atributos, algo completamente inmanejable.


#### Patrones circulares (también conocidos como Local Binary Patterns )

En esta búsqueda de patrones, se intenta ahondar un poco más en esta idea de buscar texturas.  La idea general es que para un pixel dado (en color rojo) setear una distancia y una cantidad de puntos (en color verde) a los cuales comparar con el pixel central, en cuanto a si son mas claros o mas oscuros. De esta forma se formamos patrones más grande pero sin aumentar la cantidad de posibles patrones, ya que no se tienen en cuenta todos los puntos del cuadrado y además se unen patrones simétricos (todos los semicírculos serán considerados el mismo patrón sin importar hacia qué lado apunten).

![title](./imagenes/local_binary_pattern.png)

Para lograr esto usamos la librería mahotas que implementa este patrón

In [ ]:
return mahotas.features.lbp(image, radius, points, ignore_zeros=False)

La combinación inicial de radio y cantidad de puntos que se utilizaron originalmente para las pruebas fueron respectivamente: (2,5), (2,9), (3,5), (3,9),(5,9)

### Comparaciones

Luego de generar múltiples conjuntos de atributos es hora de analizarlos.
Para esto experimentamos, usando diferentes formas de medir cuán 'buenos' son los atributos

Para las comparaciones que usan un clasificador genérico usamos nuestro primer clasificador que (como explicaremos en la sección correspondiente), está formado por un voter entre un RandomForest, un Adaboost, GradientBoost y un SVM cada uno con los mejores parámetros encontrados en el GridSearch

Es importante tener en cuenta que no todos los conjuntos de atributos tienen la misma cantidad de atributos, como muestra el siguiente gráfico.

![title](./imagenes/cant_atributos_x_batch.png)

En el mismo se ve el crecimiento exponencial del que hablamos de los patrones NxN.

#### Score de los atributos de a uno

Como primera medida se entrenó la herramienta de selección de imágenes utilizando cada atributo por separado.

![title](./imagenes/comparacion_atributos_de_a_uno.png)

Lo que se observa en el gráfico, es el score de cada atributo individualmente es muy similar, desprendiéndose ligeramente el patrón circular con 9 puntos, en un radio de 5 pixels.

#### Score de los atributos sacando de  a uno

Otro posible análisis que se hizo es el de entrenar la herramienta con todos los atributos a excepción de uno y ver como cambia el score de la herramienta.

Hay que tener en cuenta que a mayor score, peor es el conjunto de atributos (ya que si se consigue buen score sin él, significa que no estaría aportando mucha información).

![title](./imagenes/comparacion_atributos_sacando_uno.png)

Se puede observar que al igual que en el primer gráfico, hay una diferencia muy poco marcada entre todos los subconjuntos de atributos que se probaron, con una diferencia de aproximadamente 5% entre la mejor performance (habiendo sacado el histograma en blanco y negro) y la peor performance (habiendo sacado el patrón circular de 5 puntos con radio de 2 pixels)

#### Análisis univariado

El análisis univariado consiste en comparar cada atributo por separado.

Las dos formas usuales de encontrar su importancia es ver cuál es su varianza (un atributo constante no aporta ninguna información), o hacer un test de chi2 entre el atributo y la clase para evaluar la dependencia entre ambas (si el atributo es independiente o se cree independiente de la clase, no me sirve para inferir la misma).

En nuestro caso usaremos el chi2 ya que provee más información que la varianza.

Para este análisis se utilizó la PercentilSelecion de la biblioteca Sklean.feature_selection
Este método selecciona el k-percentil de variables más dependientes a la clase según el test chi2. En nuestro caso nos quedaremos con el 20-percentil 

In [ ]:
batchs = ['histogramaByN','histogramaColor','patrones2x2ByN','patrones3x3ByN','patronesCirculaesByN_2_5','patronesCirculaesByN_2_9','patronesCirculaesByN_3_9','patronesCirculaesByN_5_9','patronesCirculaesByN_3_5']
X = []
y = []
load_batch(y,path,'clases',filename) 
y = [j for i in y for j in i]
for batch in batchs:
    load_batch(X,path,batch,filename)
        
sp = SelectPercentile(chi2,20)
X_new = sp.fit_transform(X, y)

La contrapartida de este método es que se analiza comparando un atributo con la clase, perdiendo así la relación de conjunto de los atributos. Por ejemplo se puede llegar a la conclusión de que cada color por separado no aporta mucha información, pero tal vez al tener los tres colores juntos brinda muchísima información.

![title](./imagenes/univar_cant_var.png)

![title](./imagenes/univar_porcentaj_var.png)

En los gráficos anteriores podemos ver la cantidad de variables que se toman de cada extracción de atributos, que porcentaje de cada selección de atributos se utilizó.
Es importante no olvidar que cada conjunto tiene diferente cantidad de variables.
Por ejemplo en el primer gráfico parece que el patrón 2x2 o el Circular 2 5 tienen poca importancia, pero es lo opuesto, se seleccionaron todas sus atributos. 

Podemos ver por ejemplo, como el histograma de color aporta significativamente más variables que el resto de los atributos, pero si lo medimos en porcentaje solo un casi 20% de sus variables son relevantes. Además de los patrones de 2 x 2 y los patrones circulares de 5 puntos con radio de 2 pixeles, el patrón Circular 3 5 también aporta un gran porcentaje de variables.

#### Análisis multivariado

El análisis multivariado, en contraposición al análisis univariado. Evalúa a todos los atributos en conjunto, pudiendo detectar la relación entre los atributos que comentamos anteriormente.

Aprovechamos los árboles de decisión que internamente hacen este proceso al decidir por qué atributo branchear, usando entre otras opciones es minimizar la entropía o usar Gain Ratio.

En este análisis se usó el ExtraTreesClassifier de la biblioteca sklearn


In [ ]:
clf = ExtraTreesClassifier()
clf = clf.fit(X, y)
fi = clf.feature_importances_

![title](./imagenes/multivar_porcentaj_acum.png)

![title](./imagenes/multivar_porcentaj_prom.png)

En este caso nos volvemos a encontrar con el tema de los conjuntos de atributos con diferentes cardinalidades. Si tenemos en cuenta la importancia de los conjuntos como un todo, el histograma color y el patrón 3x3 son los más importantes. Pero esto sucede ya que ambos son los que tienen mayor cantidad de atributos.

En cambio si vemos la importancia relativa de cada atributo, lo de los patrones 2x2 son los más importantes.


#### Conclusiones

Habiendo realizado los análisis anteriores, podemos sacar algunas conclusiones.

Consideramos que el conjunto de atributos más importante es el de patrones 2x2, ya que como se vio tanto en el análisis univariado como multivariado, es quien tiene mayor importancia relativa. O sea sus pocos atributos concentran muchas información.


En un inicio de la experimentación pensamos que iba a ser necesario no tener en cuenta todos los conjuntos de atributos por un tema de tiempo de entrenamiento. Además se puede generar una caída de la performance debido al problema conocido y tratado en el trabajo anterior de la curse of dimensionality.

Para nuestra sorpresa nos encontramos con que no sólo no perjudica el score usar todos los atributos, sino que tampoco conlleva una diferencia de tiempo sustancial(la gran diferencia la encontramos en la cantidad de imágenes utilizadas al entrenar).

## Sesión 2:Modelos

En un primer momento pensamos un esquema donde se unían a través de un voter, una instancia de 4 algoritmos vistos (RandomForest,AdaBoosting,SVM y GradientBoosting) y cada una de estas instancias se optimizaba a sus mejores parámetros usando GridSearch.

In [ ]:
est = [RandomForest(),Boosting(),SVM(),Gradient()]
clf = VotingClassifier(estimators=est)

def Boosting():
    tuned_parameters = [{'n_estimators':[30,50,75],'learning_rate':[0.25,0.5,0.75,1]}]
    return ('Boosting',GridSearchCV(AdaBoostClassifier(), tuned_parameters,v=5,n_jobs=-1))

def RandomForest():
    tuned_parameters = {'n_estimators':[5,10,15],'max_features':['sqrt','log2',None],'bootstrap':[True,False]}]
    return ('RandomForest',GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,n_jobs=-1))

def Gradient():
    tuned_parameters = [{'loss': ['deviance', 'exponential'],'n_estimators':[75,100,150] ,'learning_rate':[0.05,0.1,0.2]}]
    return ('Gradient',GridSearchCV(GradientBoostingClassifier(), tuned_parameters, cv=5,n_jobs=-1))

def SVM():
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    return ('SVM',GridSearchCV(SVC(), tuned_parameters, cv=5,n_jobs=-1))


Al experimentar, encontramos que este esquema tardaba mucho en entrenarse y que no conseguía una muy buena performance (58%).

Encontramos que el cuello de botella del entrenamiento era el GridSearch ya que al hacer una búsqueda exhaustiva en el espacio de parámetros seleccionado tiene que entrenar muchísimas veces cada clasificador (Por ejemplo en el caso de Gradient entrenaba 18 veces).

Además nuestra hipótesis de por qué no tiene una muy buena performance, es que la selección de parámetros en el GridSearch optimiza cada clasificador por separado sin tener en cuenta el voter como un todo. No solo pudiendo generar overfitting a los datos de entrenamiento, sino que todos los clasificadores se especialicen en la detección del mismo subset de datos y que todos dejen de lado otro subset.

Por esto pasamos a un esquema donde seguimos teniendo un voter para unir los múltiples clasificadores, pero en este caso usamos múltiples instancias de los clasificadores que mejor desempeño tuvieron individualmente (GradientBoosting y SVM como comentaremos en sus respectivas secciones), pero cada instancia tiene sus parámetros fijos de antemano.

Así conseguimos un esquema que no solo se entrena más rápido que el anterior ya que cada clasificador se entrena una sola vez, sino que al aumentar la cantidad de clasificadores y la variabilidad de los mismos con los diferentes parámetros, conseguimos una mayor performance (72%). 


In [ ]:
est = [RandomForest(),Boosting()]
for i in xrange(0,10):
    est.append(Gradient(i))
for i in xrange(0,10):
    est.append(SVM(i))

clf = VotingClassifier(estimators=est)


def Boosting():
    return ('Boosting',AdaBoostClassifier())

def RandomForest():
    return ('RandomForest',RandomForestClassifier()) 

def Gradient(i=0):
    if i==0:
        return ('Gradient'+str(i),GradientBoostingClassifier(random_state=i))
    elif i==1:
        return ('Gradient'+str(i),GradientBoostingClassifier(loss='exponential',random_state=i))
    elif i==2:
        return ('Gradient'+str(i),GradientBoostingClassifier(loss='exponential',learning_rate=0.2,random_state=i))
    elif i==3:
        return ('Gradient'+str(i),GradientBoostingClassifier(learning_rate=0.2,random_state=i))
    else:
        return ('Gradient'+str(i),GradientBoostingClassifier(random_state=i))

def SVM(i=0):
    if i==0:
        return ('SVM'+str(i),SVC(random_state=i))
    elif i==1:
        return ('SVM'+str(i),SVC(kernel='linear',random_state=i))
    elif i==2:
        return ('SVM'+str(i),SVC(kernel='linear',C=100,random_state=i))
    elif i==3:
        return ('SVM'+str(i),SVC(C=100,random_state=i))
    elif i==4:
        return ('SVM'+str(i),SVC(gamma=1e-3,random_state=i))
    elif i==5:
        return ('SVM'+str(i),SVC(gamma=1e-4,random_state=i))
    else:
        return ('SVM'+str(i),SVC(random_state=i))


Una vez finalizada la etapa de extracción de atributos cabe preguntarse cómo analizar estos atributos para poder tomar una decisión sobre si la imagen pertenece al grupo de los perros o al grupo de los gatos. 

Para esto implementaron varios modelos de aprendizaje supervisados independientes, y un sistema de votación donde la catalogación de una imagen será el resultado mayoritario de la catalogación de cada modelo independiente.

### SVM

Empezamos a experimentar con SVM.

En nuestra primera experimentación observamos que las svm con kernel lineal tenían un mucho mejor desempeño que las con kernel rbf (62% vs 48%), yendo en contra de nuestra hipótesis inicial, ya que no creemos que haya una relación lineal entre los atributos seleccionados y la clase.

Después de indagar sobre las causas, nos dimos cuenta que en nuestros primeros experimentos la cantidad de imágenes que usábamos era muy pequeña (500 imágenes), en particular menor que  la cantidad de atributos que extrajimos.
Esto fue lo que generó el comportamiento que antes describimos, ya que el kernel lineal se comporta bien en estos escenario (muchos atributos en relación a la cantidad de instancias) y el rbf mal.

Cuando aumentamos la cantidad de imagenes vimos que como predijimos el desempeño se invertía, cayendo el score del kernel lineal a 52% y aumentando el de rbf a 67%


In [ ]:
def SVM(i=0):
    if i==0:
        return ('SVM'+str(i),SVC(random_state=i))
    elif i==1:
        return ('SVM'+str(i),SVC(kernel='linear',random_state=i))
    elif i==2:
        return ('SVM'+str(i),SVC(kernel='linear',C=100,random_state=i))
    elif i==3:
        return ('SVM'+str(i),SVC(C=100,random_state=i))
    elif i==4:
        return ('SVM'+str(i),SVC(gamma=1e-3,random_state=i))
    elif i==5:
        return ('SVM'+str(i),SVC(gamma=1e-4,random_state=i))
    else:
        return ('SVM'+str(i),SVC(random_state=i))

### Técnicas de Ensamble


Luego de probar con las SVM, decidimos probar el desempeño de diferentes técnicas de ensamble. Ya que la combinación de múltiples clasificadores más simples ayudan a mejorar la performance.

#### Random Forest

Empezamos probando con random forest, ya que es muy sencillo y rápido de entrenar. Pero no logramos grandes avances ni siquiera sobre la performance de las svm (tener en cuenta que hablamos de una svm vs un ensamble de árboles de decisión). Conseguimos scores de entre 60% y 62%.

En nuestro esquema final mantuvimos un RandomForest como vara de comparación para los demás algoritmos.


In [ ]:
def RandomForest():
    return ('RandomForest',RandomForestClassifier()) 

#### Boosting

A diferencia del RandomForest en los algoritmos de Boosting, los clasificadores dentro del ensamble no son independientes entre sí (esto es una de las principales razones de que sea necesario más tiempo para su entrenamiento ya que no se puede paralelizar). Los clasificadores se van entrenando en secuencia y cada nuevo clasificador es entrenado teniendo en cuenta lo ya aprendido por el acumulado.

Dentro de esta clase de algoritmos probamos dos:

-Adaboost:

Consiste en una secuencia de weak learners (clasificadores apenas mejores que la elección random) que se entrenan uno tras otro en versiones modificadas de los datos de entrenamiento. Cada nuevo clasificador se entrena sobre el set de entrenamiento pero dándole mayor peso a las instancias que no fueron correctamente clasificadas todavía.
    
Finalmente todos estos clasificadores se unen en un voter pero asignándole un peso a cada clasificador 
Usando el este algoritmo conseguimos un score de 67%, mejorando significativamente lo conseguido con RandomFores

-GradientBoost: 

GradientBoost es una generalización del método de Boosting con mayor potencia que el AdaBoost.
    
Usando este método conseguimos un score del 70%
    
Al momento de generar el ensamble de clasificadores optamos por tener muchas instancias de GradientBoost sobre AdaBoost, no solo porque tiene una mejor performance, sino porque tiene una mayor variabilidad (variando por ejemplo la loss function). Lo que ayuda a compensar los bias de los clasificadores y conseguir una mejor performance en el voter.


#### Voter

Por último armamos un voter uniendo una instancia de RandomForest y AdaBoost con 10 instancias de GradientBoost y 10 de SVM variando los parámetros de las mismas (ya que tener repetido un mismo clasificador exactamente igual no aporta a la reducción de bias, sino que solo le da más peso a la elección de ese clasificador). 

El voter consiguió una performance de 72%, aunque nuestra expectativa era que mejore aún más la performance.

Además de analizar el porcentaje de aciertos en general, podemos analizar el porcentaje de acierto y falla por clase:

 <table>
  <tr>
    <td></td>
    <td>Gato</td>
    <td>Perro</td>
  </tr>
  <tr>
    <td>Gato</td>
    <td>37,0%</td>
    <td>11,0%</td>
  </tr>
  <tr>
    <td>Perro</td>
    <td>16,4%</td>
    <td>35,6%</td>
  </tr>
</table> 

Como vemos el porcentaje de aciertos de ambas clases son muy parecidos, pero si dicienten en el porcentaje de fallos.
Es un 50% más probable equivocarse al clasificar un perro que un gato.

## Conclusión

Por un lado vimos que a nuestro clasificador le es más dificil clasificar a un perro que a un gato.

En segundo lugar, consideramos que una buena selección de atributos es uno de los pasos más importantes en los trabajos de reconocimiento de imágenes. Ya que como vimos ciertos atributos aportan información fundamental para conseguir una buena performance.

Por otro lado, creemos que el ensamble generado internamente en el GradientBoost ya es bastante potente por lo que al volver a hacer un ensamble de estos clasificadores con otros (SVM,AdaBoost), el aumento de performance es muy bajo. 

Además el voter entrena cada modelo independientemente, en cambio el GradientBoost entrena cada modelo sobre 'lo que le falta aprender'. 

## Futuros trabajos

1.Usar el algoritmo surf para generar atributos de los pixels más significativos de la imagen. Hay que tener en cuenta que no se puede usar directamente los piceles destacados por el algoritmo, ya que no habría la misma cantidad para todas las imágenes. Esto se puede resolver por ejemplo teniendo en cuenta el patrón de las vecindad de los puntos destacados y generando un histograma de patrones de 2x2 o 3x3.

2.Transformar la imagen previamente, por ejemplo,ecualizándola o usando técnicas de reducción de ruido.
